<a href="https://colab.research.google.com/github/Rachit2912/Amazon_ML_challenge/blob/main/ml_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Rachit2912/Amazon_ML_challenge.git

Cloning into 'Amazon_ML_challenge'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 2), reused 20 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 4.94 MiB | 6.70 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [5]:
import requests
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt


                                                          #  fetching image : #
def fetch_image(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        image = Image.open(BytesIO(response.content))
        return image
    except Exception as e:
        print(f"Error fetching image: {e}")
        return None

#**   EXTRACTNG TEXT FROM OCR : **#
# text = extract_text_with_easyocr(preprocessed_image)
# print(text)
import requests
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import pandas as pd
import os
# import easyocr
# os.makedirs('pre_images', exist_ok=True)
# os.makedirs('ocr_text', exist_ok=True)
!rm -rf ocr_text pre_images
!sudo apt-get install tesseract-ocr
!pip install pytesseract

                              # Function to rotate the image in different orientations
def rotate_image(image, angle):
    # Convert PIL image to OpenCV format (np.array)
    image_cv = np.array(image)
    (h, w) = image_cv.shape[:2]
    center = (w // 2, h // 2)

    # Perform the rotation
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image_cv, matrix, (w, h))

    return rotated


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# Process the image URLs from the CSV file
import pytesseract
def process_images_from_csv(csv_file):
    df = pd.read_csv(csv_file)  # Reading the CSV file

    # Create directories for preprocessed images if not already present
    if not os.path.exists('pre_images'):
        os.makedirs('pre_images')

    # Add a new column for OCR text if it doesn't exist
    if 'ocr_text' not in df.columns:
        df['ocr_text'] = ''

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # if index<start:
        #   continue

        # if index>end:
        #   break
        image_url = row['image_link']  # Assuming the column is named 'image_link'
        image = fetch_image(image_url)


        if image is not None:
            # Initialize a variable to accumulate the OCR text for all orientations
            accumulated_text = ""

            # Process the image for four different orientations
            for angle in [0, 90, 180, 270]:
                rotated_image = rotate_image(image, angle)

                # Save the preprocessed (rotated) image
                # image_save_path = f'pre_images/preprocessed_image_{index}_{angle}.jpg'
                # cv2.imwrite(image_save_path, rotated_image)

                # Extract OCR text from the rotated image
                extracted_text = pytesseract.image_to_string(rotated_image)
                # extracted_text = extract_text_with_easyocr(rotated_image)
                # extracted_text = api.ocr_url(rotated_image)
                # extracted_text = ocr_space_url(url=image_url)

                # Append the OCR text for the current orientation
                accumulated_text += f"{extracted_text}\n"

            # Update the OCR text column in the DataFrame
            df.at[index, 'ocr_text'] = accumulated_text.strip()
            df.to_csv(csv_file, index=False)
            print(f"Processed image {index} and saved OCR text for all orientations.")

        else:
            print(f"Skipping image {index} due to fetch error.")

    # Save the updated DataFrame back to the CSV file
    # df.to_csv(csv_file, index=False)
    print(f"Updated CSV file saved at {csv_file}")

# Example usage
# csv_file = '/content/Amazon_ML_challenge/dataset/sample_test.csv'  # Path to your CSV file containing the image URLs
csv_file = '/content/Amazon_ML_challenge/dataset/test.csv'
process_images_from_csv(csv_file)

In [6]:
import re
import pandas as pd

# Entity to Unit map with various short forms
entity_unit_map = {
    "width": {"centimetre", "centimetres", "cms", "cm", "foot", "feets", "fts", "'", "ft", "millimetre", "millimetres", "mms", "mm", "metres", "metre", "m", "\"", "inch", "inches", "in", "yard", "yards", "yd", "yds"},
    "depth": {"centimetre", "centimetres", "cms", "cm", "foot", "feets", "fts", "'", "ft", "millimetre", "millimetres", "mms", "mm", "metres", "metre", "m", "\"", "inch", "inches", "in", "yard", "yards", "yd", "yds"},
    "height": {"centimetre", "centimetres", "cms", "cm", "foot", "feets", "fts", "'", "ft", "millimetre", "millimetres", "mms", "mm", "metres", "metre", "m", "\"", "inch", "inches", "in", "yard", "yards", "yd", "yds"},
    "item_weight": {"milligram", "mg", "milligrams", "kilogram", "kg", "kgs", "kilograms", "microgram", "micrograms", "μg", "ug", "gram", "grams", "g", "gm", "gms", "ounce", "oz", "ton", "tonnes", "lb", "lbs", "pound", "pounds"},
    "maximum_weight_recommendation": {"milligram", "mg", "milligrams", "kilogram", "kg", "kgs", "kilograms", "microgram", "micrograms", "μg", "ug", "gram", "grams", "g", "gm", "ounce", "oz", "ton", "tonnes", "lb", "lbs", "pound", "pounds"},
    "voltage": {"millivolt", "millivolts", "mV", "kilovolt", "kilovolts", "kV", "volt", "volts", "V"},
    "wattage": {"kilowatt", "kilowatts", "kW", "watt", "watts", "W"},
    "item_volume": {
        "cubic foot", "cubic foots", "cu ft", "cu fts", "cufts", "cuft", "microlitre", "μL", "microlitres", "cup", "cups", "fluid ounce", "fluid ounces", "fl oz", "floz", "centilitre", "cl", "centilitres",
        "imperial gallon", "imperial gallons", "gal", "pint", "pints", "pt", "decilitre", "decilitres", "dl", "litre", "litres", "ltr", "l", "millilitre", "ml", "millilitres", "mls",
        "quart", "quarts", "qt", "cubic inch", "cubicinch", "cubic inches", "cubicinches", "cuin", "cu in", "gallon", "gallons"}
}

# Function to dynamically build regex patterns that handle both single values and ranges
def build_regex_patterns(entity_unit_map):
    patterns = {}
    for entity, units in entity_unit_map.items():
        # Create a regex pattern that captures both single values and ranges (e.g., 20-30)
        unit_pattern = r'|'.join(units)  # Join all units into a regex alternation pattern
        # Regex pattern that matches numbers or ranges (e.g., "20", "20-30") followed by a unit
        patterns[entity] = rf'((?:\d+\.?\d*|\d+-\d+\.?\d*)\s*({unit_pattern}))'
    return patterns

def extract_dimensions(text, entity, patterns):
    if any(e in entity for e in ['width', 'height', 'depth']):
        dimension_pattern = patterns['height']  # Assuming height, width, and depth have the same pattern
        matches = re.findall(dimension_pattern, text, re.IGNORECASE)

        if matches:
            # Parse matches to extract numeric values and units
            parsed_matches = []
            for value, unit in matches:
                numeric_value = re.sub(r'[^\d\.]', '', value)
                if numeric_value:
                    parsed_matches.append((float(numeric_value), unit))

            # Remove redundancy by converting to set and back to list
            parsed_matches = list(set(parsed_matches))

            # Sort the parsed matches in descending order (max to min)
            sorted_values = sorted(parsed_matches, key=lambda x: x[0], reverse=True)

            if len(sorted_values) == 1:
                # If only one value is found, return that value as the requested entity
                return f"{sorted_values[0][0]} {sorted_values[0][1]}" if entity in ['height', 'width', 'depth'] else None
            elif len(sorted_values) == 2:
                # If two values are found, max is height, min is width or depth
                if entity == 'height':
                    return f"{sorted_values[0][0]} {sorted_values[0][1]}"
                elif entity == 'width' or entity == 'depth':
                    return f"{sorted_values[1][0]} {sorted_values[1][1]}"
            elif len(sorted_values) == 3:
                # If three values are found, max is height, 2nd max is depth, and min is width
                if entity == 'height':
                    return f"{sorted_values[0][0]} {sorted_values[0][1]}"
                elif entity == 'depth':
                    return f"{sorted_values[1][0]} {sorted_values[1][1]}"
                elif entity == 'width':
                    return f"{sorted_values[2][0]} {sorted_values[2][1]}"

    return None


# Function to extract weights based on keywords like "net weight" and return the larger value if keywords aren't present
def extract_weight_entity(text, patterns):
    if 'item_weight' in patterns:
        pattern = patterns['item_weight']
        matches = re.findall(pattern, text, re.IGNORECASE)
        # print(matches)

        if matches:
            parsed_matches = []
            for value, unit in matches:
                if '-' in value:
                  continue
                numeric_value = re.sub(r'[^\d\.]', '', value)
                if numeric_value:
                    parsed_matches.append((float(numeric_value), unit))

            if parsed_matches:
                max_value = max(parsed_matches, key=lambda x: x[0])
                return f"{max_value[0]} {max_value[1]}"

    return None

# Function to extract item volume
def extract_item_volume(text, patterns):
    if 'item_volume' in patterns:
        pattern = patterns['item_volume']
        matches = re.findall(pattern, text, re.IGNORECASE)

        if matches:
            parsed_matches = []
            for value, unit in matches:
                numeric_value = re.sub(r'[^\d\.]', '', value)
                if numeric_value:
                    parsed_matches.append((float(numeric_value), unit))

            if parsed_matches:
                max_value = max(parsed_matches, key=lambda x: x[0])
                return f"{max_value[0]} {max_value[1]}"

    return None

# Function to extract entities (including ranges) from text
def extract_entity(text, entity, patterns):
    if entity == 'item_weight':
        return extract_weight_entity(text, patterns)

    if entity == 'maximum_weight_recommendation':
        if 'maximum_weight_recommendation' in patterns:
            pattern = patterns['maximum_weight_recommendation']
            matches = re.findall(pattern, text, re.IGNORECASE)
            # print(matches)
            if matches:
                ranges = [match for match in matches if '-' in match[0]]
                # print(ranges)
                if len(ranges)!=0:
                    # value = value.split("")
                    # ans = ranges[0][0]
                    # ans = ans.split(" ")
                    # unit = ans[1]
                    # ans = ans[0]
                    # print("here",ranges,ans,unit)
                    return  ''

                parsed_matches = []
                for value, unit in matches:
                    numeric_value = re.sub(r'[^\d\.]', '', value)
                    if numeric_value:
                        parsed_matches.append((float(numeric_value), unit))

                if parsed_matches:
                    max_value = max(parsed_matches, key=lambda x: x[0])
                    return f"{max_value[0]} {max_value[1]}"

    if entity == 'voltage':
        if 'voltage' in patterns:
            pattern = patterns['voltage']
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                parsed_matches = []
                for value, unit in matches:
                    numeric_value = re.sub(r'[^\d\.]', '', value)
                    if numeric_value:
                        parsed_matches.append((float(numeric_value), unit))

                if parsed_matches:
                    max_value = max(parsed_matches, key=lambda x: x[0])
                    return f"{max_value[0]} {max_value[1]}"

    if entity == 'wattage':
        if 'wattage' in patterns:
            pattern = patterns['wattage']
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                parsed_matches = []
                for value, unit in matches:
                    numeric_value = re.sub(r'[^\d\.]', '', value)
                    if numeric_value:
                        parsed_matches.append((float(numeric_value), unit))

                if parsed_matches:
                    max_value = max(parsed_matches, key=lambda x: x[0])
                    return f"{max_value[0]} {max_value[1]}"

    if entity in ['height', 'width', 'depth']:
        return extract_dimensions(text, entity, patterns)

    if entity == 'item_volume':
        return extract_item_volume(text, patterns)

    return None

# Example usage:
patterns = build_regex_patterns(entity_unit_map)
sample_text = "30-25 kgs 60 kgs"

# Test the dimension extraction
height = extract_entity(sample_text, 'item_weight', patterns)
# width = extract_entity(sample_text, 'width', patterns)
# depth = extract_entity(sample_text, 'depth', patterns)

# print(height)
# print(width)
# print(depth)
#

In [ ]:
## updated :

import pandas as pd
import re

# Define the unit conversion map with escaping for special characters
unit_conversion_map = {
    "centimetres": "centimetre",
    "cms": "centimetre",
    "cm": "centimetre",
    "feets": "foot",
    "fts": "foot",
    "'": "foot",
    "ft": "foot",
    "millimetres": "millimetre",
    "mms": "millimetre",
    "mm": "millimetre",
    "metres": "metre",
    "m": "metre",
    "\"": "inch",
    "inches": "inch",
    "in": "inch",
    "yards": "yard",
    "yds": "yard",
    "yd": "yard",
    "milligrams": "milligram",
    "mg": "milligram",
    "kilograms": "kilogram",
    "kgs": "kilogram",
    "kg": "kilogram",
    "micrograms": "microgram",
    "μg": "microgram",
    "ug": "microgram",
    "grams": "gram",
    "gm": "gram",
    "gms": "gram",
    "g": "gram",
    "oz": "ounce",
    "lbs": "pound",
    "lb": "pound",
    "pounds": "pound",
    "tonnes": "ton",
    "tons": "ton",
    "millivolts": "millivolt",
    "mV": "millivolt",
    "kilovolts": "kilovolt",
    "kV": "kilovolt",
    "volts": "volt",
    "V": "volt",
    "kilowatts": "kilowatt",
    "kW": "kilowatt",
    "watts": "watt",
    "W": "watt",
    "cubic foots": "cubic foot",
    "cu fts": "cubic foot",
    "cufts": "cubic foot",
    "cuft": "cubic foot",
    "microlitres": "microlitre",
    "μL": "microlitre",
    "cups": "cup",
    "fluid ounces": "fluid ounce",
    "fl":"fluid",
    "fl oz": "fluid ounce",
    "floz": "fluid ounce",
    "centilitres": "centilitre",
    "cl": "centilitre",
    "imperial gallons": "imperial gallon",
    "gal": "imperial gallon",
    "pints": "pint",
    "pt": "pint",
    "decilitres": "decilitre",
    "dl": "decilitre",
    "litres": "litre",
    "ltr": "litre",
    "l": "litre",
    "millilitres": "millilitre",
    "mls": "millilitre",
    "ml": "millilitre",
    "quarts": "quart",
    "qt": "quart",
    "cubic inches": "cubic inch",
    "cubicinches": "cubic inch",
    "cuin": "cubic inch",
    "cu in": "cubic inch",
    "gallons": "gallon"
}

def convert_units(prediction, unit_conversion_map):
    if not prediction:
        return ""

    # Explicitly handle single and double quotes
    prediction = prediction.replace("'", "foot").replace("\"", "inch")

    # Convert each short form or symbol in the prediction
    for short_form, full_form in unit_conversion_map.items():
        # Escape special characters for regex
        escaped_short_form = re.escape(short_form)
        # Replace short form and symbols with full form
        prediction = re.sub(rf'\b{escaped_short_form}\b', full_form, prediction, flags=re.IGNORECASE)

    return prediction

def process_csv(input_csv_path, output_csv_path):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(input_csv_path)

    # Build regex patterns for each entity (assuming this function exists)
    patterns = build_regex_patterns(entity_unit_map)

    # Extract and update the predictions
    df['prediction'] = df.apply(lambda row: extract_entity(str(row['ocr_text']), str(row['entity_name']), patterns), axis=1)

    # Convert short forms and symbols to full form unit names
    df['prediction'] = df['prediction'].apply(lambda pred: convert_units(pred, unit_conversion_map).lower())

    # Replace NaN or None values in 'prediction' with an empty string
    df['prediction'].fillna('', inplace=True)

    # Create a new DataFrame with only 'index' and 'prediction' columns
    output_df = df[['index', 'prediction']]

    # Save the updated DataFrame to a new CSV file
    print("index",df['index'])
    output_df.to_csv(output_csv_path, index=False)

# # Example usage
input_csv_path = '/content/Amazon_ML_challenge/dataset/test.csv'
output_csv_path = '/content/Amazon_ML_challenge/dataset/test_output.csv'
process_csv(input_csv_path, output_csv_path)

from google.colab import files
files.download('/content/Amazon_ML_challenge/dataset/test_output.csv')

